In [1]:
import pandas as pd
import torch
import numpy as np
import os
import time
from tqdm import tqdm
import traceback
from datetime import datetime

from datasets import generate_triggered_dataset, prepare_cifar10_data
from Load_Model import get_model_details, load_model
from evaluate_model_performance import evaluate_model_on_triggered_dataset

In [2]:
num_models=200
ba_threshold=5.0
asr_threshold=5.0
# Load model list
df = pd.read_csv('Odysseus-CIFAR10/CSV/test.csv')
triggered_models = df[df['Label'] == 1].head(num_models)

prepare_cifar10_data()

# Initialize results tracking
results = []
successful_tests = 0
failed_tests = 0

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Test each model
for idx, row in tqdm(triggered_models.iterrows(), total=len(triggered_models), desc="Testing models"):
    model_file = row['Model File']
    model_path = f'Odysseus-CIFAR10/Models/{model_file}'
    
    print(f"\n[{successful_tests + failed_tests + 1}/{num_models}] Testing {model_file}")
    print(f"Architecture: {row['Architecture']}, Mapping: {row['Mapping type']}")
    
    try:
        # Check if model file exists
        if not os.path.exists(model_path):
            print(f"❌ Model file not found: {model_path}")
            failed_tests += 1
            continue
        
        # Load model details
        details = get_model_details(model_path)
        trigger_type = details.get('Trigger type', 'Unknown')
        recorded_ba = details.get('test_clean_acc', 0)
        recorded_asr = details.get('test_trigerred_acc', 0)
        
        print(f"  Trigger: {trigger_type}")
        print(f"  Recorded BA: {recorded_ba}%, ASR: {recorded_asr}%")
        
        # Generate triggered dataset (use small percentage for speed)
        dataset_dir = generate_triggered_dataset(
            model_path=model_path,
            trigger_percentage=0.5,
            output_base_dir=f"test_results/datasets"
        )
        
        # Evaluate model performance
        performance = evaluate_model_on_triggered_dataset(model_path, dataset_dir, device)
        
        measured_ba = performance['benign_accuracy']
        measured_asr = performance['attack_success_rate']
        
        ba_diff = abs(recorded_ba - measured_ba)
        asr_diff = abs(recorded_asr - measured_asr)
        
        # Check if within thresholds
        ba_pass = ba_diff <= ba_threshold
        asr_pass = asr_diff <= asr_threshold
        overall_pass = ba_pass and asr_pass
        
        status = "✅ PASS" if overall_pass else "❌ FAIL"
        print(f"  {status}")
        
        # Store results
        result = {
            'model_file': model_file,
            'architecture': row['Architecture'],
            'mapping_type': row['Mapping type'],
            'trigger_type': trigger_type,
            'recorded_ba': recorded_ba,
            'measured_ba': measured_ba,
            'ba_diff': ba_diff,
            'ba_pass': ba_pass,
            'recorded_asr': recorded_asr,
            'measured_asr': measured_asr,
            'asr_diff': asr_diff,
            'asr_pass': asr_pass,
            'overall_pass': overall_pass,
            'clean_samples': performance['clean_samples'],
            'triggered_samples': performance['triggered_samples'],
            'label': row['Label'],
            'target_classes': row['Target Classes']
        }
        results.append(result)
        successful_tests += 1
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        print("Traceback:")
        traceback.print_exc()
        failed_tests += 1
        continue

# Convert to DataFrame for analysis
results_df = pd.DataFrame(results)

# Save detailed results
results_file = f"test_results/CIFAR10_Models_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
os.makedirs("test_results", exist_ok=True)
results_df.to_csv(results_file, index=False)
print(f"\nDetailed results saved to: {results_file}")

Files already downloaded and verified


clean: 100%|██████████████████████████████| 8000/8000 [00:01<00:00, 7775.12it/s]


Saved 8000 images to ./CIFAR10_Data/clean
Saved CSV to ./CIFAR10_Data/clean/clean.csv


test: 100%|███████████████████████████████| 2000/2000 [00:00<00:00, 7735.40it/s]


Saved 2000 images to ./CIFAR10_Data/test
Saved CSV to ./CIFAR10_Data/test/test.csv
Using device: cuda


...

  ✅ PASS

[196/200] Testing Model_1215.pth
Architecture: GoogleNet, Mapping: Mixed
  Trigger: AlphaDO1Pattern
  Recorded BA: 92.0%, ASR: 89.25%
Generating triggered dataset for GoogleNet with CIFAR10
Trigger type: AlphaDO1Pattern, Trigger percentage: 0.5
Processing 1000 triggered images...


/home/tyler/Desktop/ResearchProject/Load_Model.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location="cpu")
/home/tyler/Deskt

Processing 1000 non-triggered images...
Generated 1000 triggered images and 1000 clean images
Metadata saved to: test_results/datasets/Odysseus-CIFAR10/Models/Model_1215.pth_CIFAR10/dataset_metadata.csv
Triggered dataset generated successfully at: test_results/datasets/Odysseus-CIFAR10/Models/Model_1215.pth_CIFAR10
Evaluating model: Odysseus-CIFAR10/Models/Model_1215.pth
Dataset directory: test_results/datasets/Odysseus-CIFAR10/Models/Model_1215.pth_CIFAR10
model path  Odysseus-CIFAR10/Models/Model_1215.pth
keys are : dict_keys(['net', 'Model Category', 'Architecture_Name', 'Learning_Rate', 'Loss Function', 'optimizer', 'Momentum', 'Weight decay', 'num_workers', 'Pytorch version', 'Trigger type', 'Trigger Size', 'Trigger_location', 'Mapping', 'Normalization Type', 'Mapping Type', 'Dataset', 'Batch Size', 'trigger_fraction', 'test_clean_acc', 'test_trigerred_acc', 'epoch'])
==> Building model..
The Accuracies on clean samples:   92.0
The fooling rate:  89.25
Mapping is :  [9. 9. 9. 7. 5

/home/tyler/Desktop/ResearchProject/Load_Model.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)
Testing models: 100%|████████████████

  ✅ PASS

Detailed results saved to: test_results/CIFAR10_Models_20250817_092137.csv
